# Claude Agents SDK with Custom MCP Server for CSV Querying

## Overview

This notebook demonstrates how to use **Claude Agents SDK** with a custom Model Context Protocol (MCP) server to query data from a CSV file.

### What is the Claude Agents SDK?

The **Claude Agents SDK** is Anthropic's official Python framework for building production AI agents that can:
- Execute tools and commands autonomously
- Read and edit files
- Search the web
- Integrate with MCP servers for custom capabilities
- Manage multi-turn conversations with built-in context

**Key features:**
- Simple `query()` function for stateless interactions
- `ClaudeSDKClient` for stateful conversations
- Native MCP (Model Context Protocol) integration
- Support for both local (stdio) and remote (HTTP) MCP servers
- Built-in streaming responses

### MCP (Model Context Protocol)

MCP is an open standard introduced by Anthropic that standardizes how AI models interact with external tools and data sources.

**Transport Types:**
- **stdio** - Local subprocess communication (what we'll use here)
- **HTTP/SSE** - Remote server communication (for deployed services)

### What We'll Build

In this demo, we'll:
1. Use the existing CSV MCP server that provides tools to query product data
2. Connect to it using stdio transport (local subprocess)
3. Use Claude Agents SDK to interact with the data naturally through conversation

## Setup and Installation

First, let's install the required dependencies:

```bash
pip install claude-agent-sdk anthropic pandas
```

Make sure you have your Anthropic API key set:

```bash
export ANTHROPIC_API_KEY='your-api-key-here'
```

In [1]:
import asyncio
import os
from pathlib import Path
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    TextBlock,
    ResultMessage
)
import pandas as pd

# Verify API key is set
if not os.getenv('ANTHROPIC_API_KEY'):
    print("⚠️  Warning: ANTHROPIC_API_KEY not set!")
    print("Please set it with: export ANTHROPIC_API_KEY='your-key'")
else:
    print("✅ Anthropic API key is set")

✅ Anthropic API key is set


## Our Sample Data

Let's first look at the CSV data we'll be querying:

In [2]:
# Load and display the sample CSV data
df = pd.read_csv('sample_data.csv')
print(f"Total products: {len(df)}\n")
df.head(10)

Total products: 15



,product_id,product_name,category,price,stock,rating
0,1,Laptop Pro 15,Electronics,1299.99,45,4.5
1,2,Wireless Mouse,Electronics,29.99,150,4.2
2,3,Office Chair Deluxe,Furniture,349.99,30,4.7
3,4,Standing Desk,Furniture,599.99,20,4.6
4,5,USB-C Hub,Electronics,49.99,200,4.3
5,6,Mechanical Keyboard,Electronics,149.99,75,4.8
6,7,Monitor 27inch,Electronics,399.99,60,4.4
7,8,Desk Lamp LED,Furniture,79.99,100,4.1
8,9,Ergonomic Mouse Pad,Electronics,24.99,250,4.0
9,10,Webcam HD,Electronics,89.99,80,4.5


## Understanding the MCP Server

Our custom MCP server (`csv_query_mcp_server.py`) provides several tools:

1. **get_all_products()** - Retrieve all products
2. **search_products_by_category(category)** - Filter by category
3. **search_products_by_price_range(min_price, max_price)** - Filter by price
4. **get_product_by_name(product_name)** - Search by name
5. **get_top_rated_products(limit)** - Get highest-rated items
6. **get_products_in_stock(min_stock)** - Check inventory
7. **get_category_statistics()** - Get aggregated stats

The server is built using **FastMCP**, which provides a simple decorator-based API for creating MCP tools.

## Creating a Query Function with MCP Server Integration

Now let's create a helper function that connects to our MCP server via stdio transport.

### How stdio Transport Works:

1. **ClaudeAgentOptions** specifies the MCP server with stdio configuration
2. The SDK launches the Python script as a subprocess
3. Communication happens through stdin/stdout pipes
4. Tools are discovered automatically via the `tools/list` MCP endpoint
5. Tool calls are executed through `tools/call` MCP endpoint
6. Results are streamed back through the async generator

In [ ]:
async def run_agentic_search(user_query: str, verbose: bool = False):
    """
    Run a query using Claude with the CSV MCP server.
    
    Args:
        user_query: The user's question
        verbose: If True, show detailed tool usage and cost information
    """
    # Configure Claude to use the external MCP server via stdio
    options = ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        system_prompt="""
        You are a helpful product assistant with access to a product catalog.
        Use the available tools to answer questions about products, including:
        - Searching by category, price, or name
        - Finding top-rated products
        - Checking inventory levels
        - Providing category statistics
        
        Always provide clear, helpful responses based on the data.
        """,
        mcp_servers={
            "csv-query": {
                "type": "stdio",
                "command": "python",
                "args": ["csv_query_mcp_server.py"]
            }
        },
        # TODO: Remove this for production!!!!!
        permission_mode="bypassPermissions",  # Auto-approve tool usage for demo
        max_turns=10,
    )
    
    print(f"Query: {user_query}\n")
    
    # Stream the response
    assistant_text = []
    tools_used = []
    
    async for message in query(prompt=user_query, options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    assistant_text.append(block.text)
                # Track tool usage
                elif hasattr(block, 'name'):
                    tools_used.append(block.name)
        
        elif isinstance(message, ResultMessage):
            if verbose:
                print(f"\n💰 Cost: ${message.total_cost_usd:.4f}")
                if tools_used:
                    print(f"🔧 Tools used: {', '.join(set(tools_used))}")
    
    # Print final response
    if assistant_text:
        print("Response:")
        print('\n'.join(assistant_text))
    
    print("\n" + "="*70 + "\n")

## Example 1: Simple Product Search

Let's ask Claude to find products in a specific category.

In [4]:
await run_agentic_search("What electronics products do we have?")

Query: What electronics products do we have?

Response:
I'll search for electronics products in our catalog.
We have 9 electronics products in our catalog:

1. **Laptop Pro 15** - $1,299.99 (45 in stock, ⭐ 4.5)
2. **Wireless Mouse** - $29.99 (150 in stock, ⭐ 4.2)
3. **USB-C Hub** - $49.99 (200 in stock, ⭐ 4.3)
4. **Mechanical Keyboard** - $149.99 (75 in stock, ⭐ 4.8)
5. **Monitor 27inch** - $399.99 (60 in stock, ⭐ 4.4)
6. **Ergonomic Mouse Pad** - $24.99 (250 in stock, ⭐ 4.0)
7. **Webcam HD** - $89.99 (80 in stock, ⭐ 4.5)
8. **Cable Organizer** - $15.99 (300 in stock, ⭐ 4.2)
9. **Portable SSD 1TB** - $129.99 (90 in stock, ⭐ 4.7)

Our electronics range from accessories like mouse pads and cable organizers to complete computing solutions like laptops and monitors. The highest-rated item is the Mechanical Keyboard with 4.8 stars. All items are currently in stock!




## Example 2: Price Range Query

Let's search for products within a specific price range.

In [5]:
await run_agentic_search("Show me products that cost between $50 and $150")

Query: Show me products that cost between $50 and $150

Response:
I'll search for products in that price range for you.
Here are the products priced between $50 and $150:

1. **Mechanical Keyboard** - Electronics
   - Price: $149.99
   - Stock: 75 units
   - Rating: 4.8/5

2. **Desk Lamp LED** - Furniture
   - Price: $79.99
   - Stock: 100 units
   - Rating: 4.1/5

3. **Webcam HD** - Electronics
   - Price: $89.99
   - Stock: 80 units
   - Rating: 4.5/5

4. **Portable SSD 1TB** - Electronics
   - Price: $129.99
   - Stock: 90 units
   - Rating: 4.7/5

All four products are currently in stock. The Mechanical Keyboard has the highest rating at 4.8/5, while the Portable SSD 1TB is also highly rated at 4.7/5.




## Example 3: Top Rated Products

Let's find the highest-rated products.

In [6]:
await run_agentic_search("What are the top 3 highest-rated products?")

Query: What are the top 3 highest-rated products?

Response:
I'll find the top 3 highest-rated products for you.
Here are the top 3 highest-rated products:

1. **Mechanical Keyboard** (Electronics)
   - Rating: 4.8/5
   - Price: $149.99
   - Stock: 75 units available

2. **Office Chair Deluxe** (Furniture)
   - Rating: 4.7/5
   - Price: $349.99
   - Stock: 30 units available

3. **Portable SSD 1TB** (Electronics)
   - Rating: 4.7/5
   - Price: $129.99
   - Stock: 90 units available

All three products are well-stocked and highly rated by customers. The Mechanical Keyboard leads with the highest rating of 4.8, while both the Office Chair Deluxe and Portable SSD share an excellent 4.7 rating.




## Example 4: Category Statistics

Let's get aggregated statistics by category.

In [7]:
await run_agentic_search(
    "Give me a summary of our product categories with average prices and ratings"
)

Query: Give me a summary of our product categories with average prices and ratings

Response:
I'll get you a summary of the product categories with their statistics.
Here's a summary of our product categories:

## **Electronics**
- **Number of Products:** 9
- **Average Price:** $243.43
- **Average Rating:** 4.40 out of 5
- **Total Stock:** 1,250 units

## **Furniture**
- **Number of Products:** 6
- **Average Price:** $215.82
- **Average Rating:** 4.45 out of 5
- **Total Stock:** 490 units

**Key Insights:**
- Furniture products have a slightly higher average rating (4.45) compared to Electronics (4.40)
- Both categories are similarly priced, with Electronics averaging about $28 more per item
- Electronics has significantly more inventory available (1,250 vs 490 units)
- We have 15 total products across both categories




## Example 5: Complex Multi-Step Query

Claude can use multiple tools in sequence to answer complex questions. Let's enable verbose mode to see the details.

In [10]:
await run_agentic_search(
    """
    I need to buy some office equipment. Can you help me find:
    1. A good keyboard (check ratings)
    2. Any furniture items under $200
    3. Tell me if they're in stock
    """,
    verbose=True
)

Query: 
    I need to buy some office equipment. Can you help me find:
    1. A good keyboard (check ratings)
    2. Any furniture items under $200
    3. Tell me if they're in stock
    


💰 Cost: $0.0237
🔧 Tools used: mcp__csv-query__search_products_by_price_range, mcp__csv-query__search_products_by_category
Response:
I'll help you find office equipment! Let me search for keyboards, furniture items under $200, and check their stock status.
Perfect! I found some great options for you. Here's what I recommend:

## 1. **Keyboard Recommendation** ✓
**Mechanical Keyboard** - $149.99
- ⭐ **Rating: 4.8/5** (Excellent!)
- 📦 **In Stock: 75 units**
- This is your best option for a keyboard with outstanding ratings

## 2. **Furniture Items Under $200** 
I found several furniture options under your budget:

### Top Picks:
1. **Bookshelf** - $189.99
   - ⭐ Rating: 4.3/5
   - 📦 **In Stock: 40 units**

2. **Desk Lamp LED** - $79.99
   - ⭐ Rating: 4.1/5
   - 📦 **In Stock: 100 units**

3. **Laptop St

## How It Works: Behind the Scenes

### MCP Protocol Flow with Claude Agents SDK

When you run a query:

1. **Server Initialization**
   - SDK reads the `mcp_servers` configuration from `ClaudeAgentOptions`
   - Launches `csv_query_mcp_server.py` as a subprocess
   - Establishes stdin/stdout communication pipes

2. **Tool Discovery**
   - SDK calls the MCP `tools/list` endpoint
   - Server returns all available tools with their schemas
   - Tools are namespaced as `mcp__csv-query__tool_name`

3. **Agent Reasoning**
   - User query is sent to Claude (via Anthropic API)
   - Claude sees the available tools in its context
   - Claude decides which tool(s) to call based on the query

4. **Tool Execution**
   - SDK sends tool call request to MCP server via `tools/call`
   - Server executes the tool (e.g., queries the CSV)
   - Results are returned through the pipe

5. **Response Generation**
   - Tool results are added to the conversation context
   - Claude generates a natural language response
   - Final output is streamed through the async generator

### Key Concepts

**query() Function:**
- Stateless, one-shot interactions
- Returns an async generator of messages
- Automatically handles the agent loop (tool calls + responses)

**ClaudeAgentOptions:**
- Configures model, system prompt, tools, and MCP servers
- `permission_mode` controls tool approval:
  - `"default"` - asks for permission
  - `"bypassPermissions"` - auto-approves all
  - `"acceptEdits"` - auto-approves edits only

**Message Types:**
- `AssistantMessage` - Contains Claude's response and tool uses
- `TextBlock` - Text content from Claude
- `ToolUseBlock` - Tool call information
- `ResultMessage` - Final result with cost and metadata

## Comparison: query() vs ClaudeSDKClient

The Claude Agents SDK provides two main ways to interact with Claude:

### query() - Stateless (What we used above)
```python
async for message in query(prompt="Your question", options=options):
    # Process messages
    pass
```
- Best for one-off questions
- No conversation history
- Simplest API

### ClaudeSDKClient - Stateful
```python
async with ClaudeSDKClient(options=options) as client:
    await client.query("First question")
    async for msg in client.receive_response():
        print(msg)
    
    # Continue conversation with context
    await client.query("Follow-up question")
    async for msg in client.receive_response():
        print(msg)
```
- Maintains conversation history
- Multi-turn conversations
- More control over the flow

## Example 6: Stateful Conversation with ClaudeSDKClient

Let's demonstrate a multi-turn conversation where context is maintained.

In [ ]:
from claude_agent_sdk import ClaudeSDKClient

async def stateful_conversation():
    """Demonstrate a stateful multi-turn conversation."""
    options = ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        system_prompt="You are a helpful product assistant.",
        mcp_servers={
            "csv-query": {
                "type": "stdio",
                "command": "python",
                "args": ["csv_query_mcp_server.py"]
            }
        },
        permission_mode="bypassPermissions",
    )
    
    async with ClaudeSDKClient(options=options) as client:
        # First query
        print("User: What's the highest-rated product?\n")
        await client.query("What's the highest-rated product?")
        
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}\n")
        
        # Follow-up query (uses context from previous)
        print("User: Is it in stock?\n")
        await client.query("Is it in stock?")
        
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}\n")
        
        # Another follow-up
        print("User: What other products are in the same category?\n")
        await client.query("What other products are in the same category?")
        
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}\n")

await stateful_conversation()

## Key Takeaways

### Benefits of Claude Agents SDK with MCP

1. **Simplicity** - Clean async API with streaming responses
2. **Flexibility** - Works locally (stdio) or remotely (HTTP)
3. **Standardization** - MCP protocol for all tool integrations
4. **Control** - Fine-grained permission modes and configuration

### When to Use stdio vs. HTTP

**stdio (Local)**:
- Development and testing
- Single-user applications
- No network required
- Process isolation

**HTTP (Remote)**:
- Production deployments
- Multi-user services
- Centralized tool management
- Cross-network access

### Best Practices

1. **Tool Design**
   - Keep tools focused and single-purpose
   - Provide clear descriptions
   - Return structured, parseable data

2. **Error Handling**
   - Return meaningful error messages
   - Handle edge cases gracefully
   - Validate inputs

3. **Performance**
   - Cache expensive operations
   - Optimize data queries
   - Consider async operations

4. **Security**
   - Validate and sanitize inputs
   - Use appropriate permission modes
   - Limit file system access

## Next Steps

Now that you understand the basics, you can:

1. **Extend the MCP Server**
   - Add more tools (insert, update, delete)
   - Support multiple CSV files
   - Add data validation

2. **Build Custom Tools**
   - Create in-process SDK tools with `@tool` decorator
   - Combine multiple MCP servers
   - Mix SDK and external servers

3. **Deploy as HTTP Server**
   - Convert to HTTP transport
   - Add authentication
   - Scale horizontally

4. **Build Multi-Agent Systems**
   - Define custom agents in options
   - Implement agent handoffs
   - Orchestrate complex workflows

### Resources

- [Claude Agents SDK Documentation](https://platform.claude.com/docs/en/agent-sdk/overview)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [FastMCP Documentation](https://github.com/modelcontextprotocol/python-sdk)
- [Claude API Documentation](https://docs.anthropic.com/)